In [1]:
from langgraph.graph import StateGraph, MessagesState, START, END
from IPython.display import Image, display
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage,SystemMessage, ToolMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from tavily import TavilyClient
from typing import Literal

d:\Python\Python314\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


ModuleNotFoundError: No module named 'langchain_google_genai'

In [48]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your TAVILY API key: ")
    

# TOOLS

In [49]:
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

@tool
def internet_search(
    query: str,
    max_results: int = 5,
    start_date: str = None,
    end_date: str = None,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a tavily web search
    Args:
        query: The search query. str
        max_results: The maximum number of results to return. int
        start_date: The start date for the search in YYYY-MM-DD format. str
        end_date: The end date for the search in YYYY-MM-DD format. str
        topic: The topic of the search. Can be "general", "news", "finance". str
        include_raw_content: Whether to include the raw content of the results. boolean
    """
    return tavily_client.search(
        query,
        max_results=max_results,
        start_date= start_date,
        end_date= end_date,
        include_raw_content=include_raw_content,
        topic=topic,
    )

# LLM

In [50]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    
)
tools = [internet_search]
tools_by_name = {tool.name: tool for tool in tools}
llmwithtools = llm.bind_tools(tools)

# NODE

In [51]:
def call_llm_node(state: MessagesState):
    
    messages = state["messages"]
    print(f"message = {messages}")
    print(f"lstmessage = {messages[-1]}")
    response = llmwithtools.invoke(messages)
    return {"messages": [response]}

In [52]:
def execute_tool_calls_node(state: MessagesState):
    last_message = state["messages"][-1]
    tool_outputs = []
    for tool_call in last_message.tool_calls:
        if tool_call["name"] == "internet_search":
            result = internet_search.invoke(tool_call["args"])
            tool_outputs.append(ToolMessage(tool_call_id=tool_call['id'], content=str(result)))
        # Add other tool handling as needed
    return {"messages": tool_outputs}


In [53]:
def should_call_tools(state: MessagesState):
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "tools"
    else:
        return "end"

# GRAPH

In [54]:
graph = StateGraph(MessagesState)
graph.add_node("llm_node", call_llm_node)
graph.add_node("execute_tool_calls_node", execute_tool_calls_node)

graph.add_edge(START,"llm_node")
graph.add_conditional_edges(
    "llm_node", 
    should_call_tools,
    {
        "tools": "execute_tool_calls_node",
        "end": END,
    }
)
graph.add_edge("execute_tool_calls_node", "llm_node")


checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": "SigmaBoy1"}}

In [ ]:
# use the compiled app (CompiledStateGraph) which exposes the graph
display(Image(app.get_graph().draw_mermaid_png()))

# RESULTS

In [55]:
input1 = {"messages": [SystemMessage(content="You are a helpful Agentic AI. You have tools use as per need."), HumanMessage(content="oki")]}

In [56]:
results = app.invoke(input1, config)

print("Final result:", results)
ai_msg = results["messages"][-1]
ai_msg.pretty_print()

message = [SystemMessage(content='You are a helpful Agentic AI. You have tools use as per need.', additional_kwargs={}, response_metadata={}, id='f0befd34-646b-4b3a-a650-27541b983368'), HumanMessage(content='oki', additional_kwargs={}, response_metadata={}, id='e3ed84b0-e1a9-4a47-90a3-cc3b87a0a232')]
lstmessage = content='oki' additional_kwargs={} response_metadata={} id='e3ed84b0-e1a9-4a47-90a3-cc3b87a0a232'
Final result: {'messages': [SystemMessage(content='You are a helpful Agentic AI. You have tools use as per need.', additional_kwargs={}, response_metadata={}, id='f0befd34-646b-4b3a-a650-27541b983368'), HumanMessage(content='oki', additional_kwargs={}, response_metadata={}, id='e3ed84b0-e1a9-4a47-90a3-cc3b87a0a232'), AIMessage(content='How can I help you? ', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id

In [45]:
async for event in app.astream_events(input1, config):
    if event["event"] == "on_chat_model_stream":
        chunk = event["data"]["chunk"].content
        if chunk:
            print(chunk, end="", flush=True)


message = [SystemMessage(content='You are a helpful Agentic AI. You have tools use as per need.', additional_kwargs={}, response_metadata={}, id='d930e449-7e7c-427b-84a1-79e6b9cee886'), HumanMessage(content='hello summarize this chat', additional_kwargs={}, response_metadata={}, id='7c0c9268-942e-4755-8b6a-7f1bfc1996fe'), AIMessage(content="I need more context to summarize our chat. Please provide me with the previous messages or tell me what you'd like me to focus on.", additional_kwargs={}, response_metadata={'safety_ratings': [], 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'model_provider': 'google_genai'}, id='lc_run--85df9c81-eddc-4c06-820d-fde76b8b9470', usage_metadata={'input_tokens': 278, 'output_tokens': 29, 'total_tokens': 307, 'input_token_details': {'cache_read': 0}}), SystemMessage(content='You are a helpful Agentic AI. You have tools use as per need.', additional_kwargs={}, response_metadata={}, id='0a753045-4d4c-4500-ac3d-250bc6ba0294'), HumanMessage(

In [ ]:
lst = results["messages"][-1]
for tool_call in lst.tool_calls:
    print(tool_call['id'])

# CHECKPOINT TESTING

In [ ]:
# checkpointer = MemorySaver()
# config = {"configurable": {"thread_id": "SigmaBoy_2"}}
latest_checkpoint = checkpointer.get(config)
messages = latest_checkpoint["channel_values"]["messages"]
print(messages)
last_ai_msg = None
for msg in reversed(messages):
    if msg.__class__.__name__ == "AIMessage":
        last_ai_msg = msg.content
        break
print("*"*100)
print(last_ai_msg)